## Chapter 3 - word2vec
---

In [15]:
import numpy as np
import matplotlib.pyplot as plt

##### 1. 학습 데이터 준비


1. 말뭉치 생성하는 함수

In [2]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

2. 맥락과 타깃 생성하는 함수

In [3]:
def create_contexts_target(corpus, window_size=1):
    '''맥락과 타깃 생성

    :param corpus: 말뭉치(단어 ID 목록)
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return:
    '''
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

3. 맥락을 원 핫 벡터 표현으로 바꾸기

In [4]:
def convert_one_hot(corpus, vocab_size):
    '''원핫 표현으로 변환

    :param corpus: 단어 ID 목록(1차원 또는 2차원 넘파이 배열)
    :param vocab_size: 어휘 수
    :return: 원핫 표현(2차원 또는 3차원 넘파이 배열)
    '''
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot

##### 2. 간단한 CBOW 모델 구현하기

원래 책에는 책 전용 구현 규칙을 따른다. 하지만 이미 파이토치를 공부해서 CNN을 만들어 봤기에... 책에 있는 내용들을 파이토치로 다시 구현해보겠다

1. 모델 구현

In [5]:
import torch
import torch.nn as nn

class SimpleCBOW(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        V = vocab_size
        H = hidden_size
        
        # 계층
        self.in_layer = nn.Linear(V,H)
        self.out_layer = nn.Linear(H,V)

        self.word_vecs = self.in_layer.weight # 입력 측의 가중치를 단어의 분산 표현으로 사용할 것이다!

    def forward(self, contexts):
        h0 = self.in_layer(contexts[0,:])
        h1 = self.in_layer(contexts[1,:])
        h = (h0 + h1) * 0.5 # 계층이 아니더라도 이렇게 연산을 해주면 텐서가 메모리에서 지워지지 않는 한, 계산 그래프는 계속해서 생성될 것이다. 물론  __init__에 임의의 계층을 즉석으로 만들어서 붙여놔도 된다. 
        # 이거 헷갈리면 계층과 함수에 대해서 다시 공부하도록 하자, 이는 내 깃허브 밑바닥부터 시작하는 딥러닝 3에 질문과 답변에 잘 적어두었다. 그리고 파이토치 공부하기 nn.Module도 참고하자
        out = self.out_layer(h)
        return out


2. 데이터셋 준비

In [20]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import time

text = 'Yoy say goodbye and I say hello.'


class Corpus_Dataset(Dataset):
    def __init__(self, text, window_size = 1):
        self.window_size = window_size
        self.corpus, self.word_to_id, self.id_to_word = preprocess(text) # 먼저 텍스트를 말뭉치와 word_to_id로 바꾼다
        self.vocab_size = len(self.word_to_id)

        contexts, target = create_contexts_target(self.corpus, self.window_size)
        self.contexts = convert_one_hot(contexts, self.vocab_size)
        self.target = convert_one_hot(target, self.vocab_size)
        self.target = target

    def __len__(self):
        return len(self.contexts) # 데이터의 갯수 반환 형상이 (6,2,7)이면 데이터의 갯수는 6이다.

    def __getitem__(self, idx): # 데이터를 몇개씩 뽑을거냐 인데 저렇게 해두면 데이터가 올바르게 추출된다
        input = torch.tensor(self.contexts[idx], dtype = torch.float32)
        label = torch.tensor(self.target[idx], dtype = torch.long) # 손실함수 구할때 라벨은 long로 해야 한다고 하더라...
        return input, label


In [21]:
from torch.utils.data import DataLoader

dataset = Corpus_Dataset(text)
dataLoader = DataLoader(dataset)

for input, label in dataLoader:
    print(label)
    print(input)


tensor([1])
tensor([[[1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0.]]])
tensor([2])
tensor([[[0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.]]])
tensor([3])
tensor([[[0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0.]]])
tensor([4])
tensor([[[0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.]]])
tensor([1])
tensor([[[0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.]]])
tensor([5])
tensor([[[0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1.]]])


3. 모델 설정

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #  torch.cuda.is_available() GPU를 사용가능하면 True, 아니라면 False를 리턴
print("지금 사용하는 device :",device)

model = SimpleCBOW(vocab_size = 7, hidden_size = 5)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.05)

dataset = Corpus_Dataset(text)
dataloader = DataLoader(dataset)

지금 사용하는 device : cuda:0


In [23]:
dataset = Corpus_Dataset(text)
dataLoader = DataLoader(dataset)

for inputs, labels in dataLoader:
    inputs.to(device)
    labels.to(device)
    model.to(device)

    output = model(inputs.squeeze().to(device))
    loss = criterion(output, labels.squeeze().to(device))
    optimizer.zero_grad() # 기울기를 0으로 설정
    print(output)
    _, preds = torch.max(output, 0) # 결과값 추출

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

4. 학습하기

In [ ]:
def train_model(model, dataloaders, crtierion, optimizer, device, num_epochs = 10):
    since = time.time()
    acc_history = [] # 각 에폭마다 정확도 저장
    loss_history = [] # 각 에폭마다 손실함수값 저장
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        running_loss = 0.0 # 손실 함수
        running_corrects = 0 # 정답갯수

        for inputs, labels in dataloaders:
            inputs.requires_grad_(True)
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.to(device)

            outputs = model(inputs.squeeze()) # 순전파
            
            loss = crtierion(outputs, labels.squeeze()) 
            optimizer.zero_grad() # 기울기를 0으로 설정
            # _, preds = torch.max(outputs, 1) # 결과값 추출 배치라면 이게 맞는데 아니라면 출력이 1x7 라서 오류가 일어남
            _, preds = torch.max(outputs, 0)
            loss.backward() # 역전파 실행 이때 requires_grad = True가 된 완전연결층만 역전파가 됨
            optimizer.step() # 기울기 업데이트

            running_loss += loss.item() * inputs.size(0) # 출력 결과와 레이블의 오차를 계산한 결과를 누적하여 저장한다
            # loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있습니다.
            running_corrects += torch.sum(preds == labels.data) # 출력 결과와 레이블이 동일한지 확인한 결과를 누적하여 저장

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss,epoch_acc)) # 참고) {:.소수점 자리수f} 포맷 코드 : {} 내에 실수의 소수점 자리수(.소수점 자리수f)를 지정해 줄 수 있음 소수점 4자리 까지 표시함
        if epoch_acc > best_acc: # 만약에 어떤 에폭에서의 정확도가 최고 정확도보다 높을 경우 업데이트
            best_acc = epoch_acc
        
        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since # 실행 시간 계산
    print(f'실행 시간 : {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'이 모델의 최고 정확도: {best_acc:4f}') # 최고 정확도 

    return acc_history, loss_history



Epoch 0/999
----------


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
acc_history, loss_history = train_model(model, dataloader, criterion, optimizer, device)

In [ ]:
plt.plot(acc_history)
plt.plot(loss_history)